<a href="https://colab.research.google.com/github/GaryM02/applied_predictive_analytics/blob/main/PredAnalyticsProj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks/PredictiveAnalyticsProject')

# **Import Skin diseases image dataset From Kaggle**

In [ ]:
import kagglehub

# Download the dataset
path = kagglehub.dataset_download("ismailpromus/skin-diseases-image-dataset")

print("Path to dataset files:", path)

100%|██████████| 5.19G/5.19G [01:23<00:00, 66.9MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/ismailpromus/skin-diseases-image-dataset/versions/1


In [ ]:
import os
import shutil

root_path = '/root/.cache/'
remove_folder = "kagglehub"

for item in os.listdir(root_path):
    item_path = os.path.join(root_path, item)

    # Skip the "Colab Notebooks" folder
    if item == remove_folder:
        try:
          if os.path.isfile(item_path):
              os.remove(item_path)  # Delete file
              print(f"Deleted file: {item_path}")
          elif os.path.isdir(item_path):
              shutil.rmtree(item_path)  # Delete folder and all its contents
              print(f"Deleted folder: {item_path}")
        except Exception as e:
            print(f"Error deleting {item_path}: {e}")


Deleted folder: /root/.cache/kagglehub


In [ ]:
from typing_extensions import Counter
def get_directory_size(path):
    total_size = 0
    for dirpath, _, filenames in os.walk(path):
        for f in filenames:
            fp = os.path.join(dirpath, f)
            total_size += os.path.getsize(fp)
    return total_size

# inspect data folder
inspect_path_root = "/root/.cache/kagglehub/datasets/ismailpromus/skin-diseases-image-dataset/versions/1/IMG_CLASSES"
inspect_path_in_drive = "/content/drive/MyDrive/Colab Notebooks/PredictiveAnalyticsProject/data/train"
inspect_path = inspect_path_root

count = 0
for path in os.listdir(inspect_path):
    full_path = os.path.join(inspect_path, path)
    print(full_path)
    if os.path.isdir(full_path):
        # Count the number of items in the directory

        num_files = len(os.listdir(full_path))
        count += num_files
        # Calculate the total memory in MB
        size_in_mb = get_directory_size(full_path) / (1024 * 1024)
        print(f"{path}: {num_files} files, {size_in_mb:.2f} MB")

print(f"Total: {count} files")

/root/.cache/kagglehub/datasets/ismailpromus/skin-diseases-image-dataset/versions/1/IMG_CLASSES/10. Warts Molluscum and other Viral Infections - 2103
10. Warts Molluscum and other Viral Infections - 2103: 0 files, 0.00 MB
/root/.cache/kagglehub/datasets/ismailpromus/skin-diseases-image-dataset/versions/1/IMG_CLASSES/4. Basal Cell Carcinoma (BCC) 3323
4. Basal Cell Carcinoma (BCC) 3323: 0 files, 0.00 MB
/root/.cache/kagglehub/datasets/ismailpromus/skin-diseases-image-dataset/versions/1/IMG_CLASSES/6. Benign Keratosis-like Lesions (BKL) 2624
6. Benign Keratosis-like Lesions (BKL) 2624: 0 files, 0.00 MB
/root/.cache/kagglehub/datasets/ismailpromus/skin-diseases-image-dataset/versions/1/IMG_CLASSES/1. Eczema 1677
1. Eczema 1677: 0 files, 0.00 MB
/root/.cache/kagglehub/datasets/ismailpromus/skin-diseases-image-dataset/versions/1/IMG_CLASSES/9. Tinea Ringworm Candidiasis and other Fungal Infections - 1.7k
9. Tinea Ringworm Candidiasis and other Fungal Infections - 1.7k: 0 files, 0.00 MB
/roo

# **Vision Transformer - Processing Images**
- Resize and split into patches
- Create image transformation pipeline
- Load images and apply transformation
- Split into patches

In [ ]:
import os
import shutil
import random

def split_dataset(data_path, output_dir, train_ratio=0.7, val_ratio=0.15, test_ratio=0.15):
    # Check that the ratios add up to 1.0
    assert train_ratio + val_ratio + test_ratio == 1.0, "Ratios must sum to 1.0"

    # # Define directories for train, val, and test
    # for split in ['train', 'validation', 'test']:
    #     split_path = os.path.join(output_dir, split)
    #     os.makedirs(split_path, exist_ok=True)

    # Iterate over each class folder
    for class_name in os.listdir(data_path):
        class_path = os.path.join(data_path, class_name)
        print(class_path)

        if not os.path.isdir(class_path):
            continue  # Skip any non-directory files

        # create class dirs in train, val, test paths
        train_path = os.path.join(output_dir, 'train', class_name)
        os.makedirs(train_path, exist_ok=True)
        val_path = os.path.join(output_dir, 'validation', class_name)
        os.makedirs(val_path, exist_ok=True)
        test_path = os.path.join(output_dir, 'test', class_name)
        os.makedirs(test_path, exist_ok=True)

        # List all images in the class directory and shuffle
        images = os.listdir(class_path)
        print(len(images))

        # Calculate split sizes
        train_size = int(len(images) * train_ratio)
        val_size = int(len(images) * val_ratio)
        test_size = len(images) - train_size - val_size

        # Split the images
        train_images = images[:train_size]
        val_images = images[train_size:train_size + val_size]
        test_images = images[train_size + val_size:]
        print(len(train_images), len(val_images), len(test_images))

        # Move images to their respective splits
        for image in train_images:
            shutil.move(os.path.join(class_path, image), train_path)


        for image in val_images:
            shutil.move(os.path.join(class_path, image), val_path)


        for image in test_images:
            shutil.move(os.path.join(class_path, image), test_path)


    print(f"Dataset has been split into train, validation, and test sets in: {data_path}")


In [ ]:
os.chdir('/content/drive/MyDrive/Colab Notebooks/PredictiveAnalyticsProject')
!pwd

/content/drive/MyDrive/Colab Notebooks/PredictiveAnalyticsProject


In [ ]:

split_dataset("/root/.cache/kagglehub/datasets/ismailpromus/skin-diseases-image-dataset/versions/1/IMG_CLASSES", "data", train_ratio=0.7, val_ratio=0.15, test_ratio=0.15)

/root/.cache/kagglehub/datasets/ismailpromus/skin-diseases-image-dataset/versions/1/IMG_CLASSES/10. Warts Molluscum and other Viral Infections - 2103
2100
1470 315 315
/root/.cache/kagglehub/datasets/ismailpromus/skin-diseases-image-dataset/versions/1/IMG_CLASSES/4. Basal Cell Carcinoma (BCC) 3323
3320
2324 498 498
/root/.cache/kagglehub/datasets/ismailpromus/skin-diseases-image-dataset/versions/1/IMG_CLASSES/6. Benign Keratosis-like Lesions (BKL) 2624
2076
1453 311 312
/root/.cache/kagglehub/datasets/ismailpromus/skin-diseases-image-dataset/versions/1/IMG_CLASSES/1. Eczema 1677
1674
1171 251 252
/root/.cache/kagglehub/datasets/ismailpromus/skin-diseases-image-dataset/versions/1/IMG_CLASSES/9. Tinea Ringworm Candidiasis and other Fungal Infections - 1.7k
1699
1189 254 256
/root/.cache/kagglehub/datasets/ismailpromus/skin-diseases-image-dataset/versions/1/IMG_CLASSES/8. Seborrheic Keratoses and other Benign Tumors - 1.8k
1844
1290 276 278
/root/.cache/kagglehub/datasets/ismailpromus/ski

In [ ]:
# prompt: get all the classnames

import os

def get_classnames(data_path):
    """
    Gets all the class names (directory names) within the given data path.
    """
    classnames = []
    for class_name in os.listdir(data_path):
        class_path = os.path.join(data_path, class_name)
        if os.path.isdir(class_path):
            classnames.append(class_name)
    return classnames

# Example usage (assuming you have a data_path variable):
classnames = get_classnames(data_path)

['10. Warts Molluscum and other Viral Infections - 2103',
 '4. Basal Cell Carcinoma (BCC) 3323',
 '6. Benign Keratosis-like Lesions (BKL) 2624',
 '1. Eczema 1677',
 '9. Tinea Ringworm Candidiasis and other Fungal Infections - 1.7k',
 '8. Seborrheic Keratoses and other Benign Tumors - 1.8k',
 '7. Psoriasis pictures Lichen Planus and related diseases - 2k',
 '2. Melanoma 15.75k',
 '3. Atopic Dermatitis - 1.25k',
 '5. Melanocytic Nevi (NV) - 7970']

In [ ]:
# change classnames
classnames = [
    '10. Warts Molluscum and other Viral Infections - 2103',
    '4. Basal Cell Carcinoma (BCC) 3323',
    '6. Benign Keratosis-like Lesions (BKL) 2624',
    '1. Eczema 1677',
    '9. Tinea Ringworm Candidiasis and other Fungal Infections - 1.7k',
    '8. Seborrheic Keratoses and other Benign Tumors - 1.8k',
    '7. Psoriasis pictures Lichen Planus and related diseases - 2k',
    '2. Melanoma 15.75k',
    '3. Atopic Dermatitis - 1.25k',
    '5. Melanocytic Nevi (NV) - 7970'
]

new_classnames = [
    'Viral Infections',
    'Basal Cell Carcinoma',
    'Benign Keratosis-like Lesions',
    'Eczema',
    'Fungal Infection',
    'Benign Tumor',
    'Psoriasis',
    'Melanoma',
    'Atopic Dermatitis',
    'Melanocytic Nevi'
]

In [ ]:
# find directory where in classnames and update it
def update_classnames(data_path, old_classnames, new_classnames):
    """
    Updates the class names in the given data path to match the new class names.
    """
    for old_name, new_name in zip(old_classnames, new_classnames):
        old_path = os.path.join(data_path, old_name)
        new_path = os.path.join(data_path, new_name)
        os.rename(old_path, new_path)
        print(f"Renamed to {new_name}")

# Example usage (assuming you have a data_path variable):
test_data_path = "/content/drive/MyDrive/Colab Notebooks/PredictiveAnalyticsProject/data/test"
train_data_path = "/content/drive/MyDrive/Colab Notebooks/PredictiveAnalyticsProject/data/train"
validation_data_path = "/content/drive/MyDrive/Colab Notebooks/PredictiveAnalyticsProject/data/validation"

for path in [test_data_path, train_data_path, validation_data_path]:
  update_classnames(path, classnames, new_classnames)

Renamed to Viral Infections
Renamed to Basal Cell Carcinoma
Renamed to Benign Keratosis-like Lesions
Renamed to Eczema
Renamed to Fungal Infection
Renamed to Benign Tumor
Renamed to Psoriasis
Renamed to Melanoma
Renamed to Atopic Dermatitis
Renamed to Melanocytic Nevi
Renamed to Viral Infections
Renamed to Basal Cell Carcinoma
Renamed to Benign Keratosis-like Lesions
Renamed to Eczema
Renamed to Fungal Infection
Renamed to Benign Tumor
Renamed to Psoriasis
Renamed to Melanoma
Renamed to Atopic Dermatitis
Renamed to Melanocytic Nevi
Renamed to Viral Infections
Renamed to Basal Cell Carcinoma
Renamed to Benign Keratosis-like Lesions
Renamed to Eczema
Renamed to Fungal Infection
Renamed to Benign Tumor
Renamed to Psoriasis
Renamed to Melanoma
Renamed to Atopic Dermatitis
Renamed to Melanocytic Nevi


# ***Creating Utility Functions To Fine Tune Both Models***

In [3]:
import os
import time

import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.amp import autocast, GradScaler
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms
from torchvision.models import EfficientNet_B0_Weights
import torchvision.models as models
from tqdm.auto import tqdm
from torch.optim import AdamW
from torch.optim.lr_scheduler import ReduceLROnPlateau

import numpy as np
from torchvision.transforms import RandAugment
from transformers import ViTImageProcessor


In [4]:
# Required constants.
# Paths and basic settings
BATCH_SIZE = 128
NUM_WORKERS = 8
device = "cuda" if torch.cuda.is_available() else "cpu"
epochs = 50
ROOT_DIR = "data"
IMAGE_SIZE = 224

In [5]:
class EarlyStopping:
    """
    Early stops the training if validation loss doesn't improve after a given patience.
    """
    def __init__(self, patience=7, verbose=False, delta=0, path='checkpoint.pt', trace_func=print):
        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
                            Default: 7
            verbose (bool): If True, prints a message for each validation loss improvement.
                            Default: False
            delta (float): Minimum change in the monitored quantity to qualify as an improvement.
                            Default: 0
            path (str): Path for the checkpoint to be saved to.
                            Default: 'checkpoint.pt'
            trace_func (function): Trace print function.
                            Default: print
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path
        self.trace_func = trace_func

    def __call__(self, val_loss, model):
        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            self.trace_func(f"EarlyStopping counter: {self.counter} out of {self.patience}")
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''Saves model when validation loss decreases.'''
        if self.verbose:
            self.trace_func(f"Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...")
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss


def save_plots(train_acc, valid_acc, train_loss, valid_loss, pretrained):
    """
    Function to save the loss and accuracy plots to disk with improved performance.
    """

    # Convert lists to numpy arrays for faster processing
    train_acc = np.array(train_acc)
    valid_acc = np.array(valid_acc)
    train_loss = np.array(train_loss)
    valid_loss = np.array(valid_loss)

    # Use a single figure and axes to plot both accuracy and loss, reducing overhead
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

    # Plot accuracy
    ax1.plot(train_acc, color="green", linestyle="-", label="Train Accuracy")
    ax1.plot(valid_acc, color="blue", linestyle="-", label="Validation Accuracy")
    ax1.set_xlabel("Epochs")
    ax1.set_ylabel("Accuracy")
    ax1.set_title("Training and Validation Accuracy")
    ax1.legend()

    # Plot loss
    ax2.plot(train_loss, color="orange", linestyle="-", label="Train Loss")
    ax2.plot(valid_loss, color="red", linestyle="-", label="Validation Loss")
    ax2.set_xlabel("Epochs")
    ax2.set_ylabel("Loss")
    ax2.set_title("Training and Validation Loss")
    ax2.legend()

    # Save the figure in one go
    fig.savefig(f"metrics_pretrained_{pretrained}.png")
    plt.close(fig)

# Image normalization transforms.
def normalize_transform(pretrained):
    if pretrained:  # Normalization for pre-trained weights.
        normalize = transforms.Normalize(
            mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
        )
    else:  # Normalization when training from scratch.
        normalize = transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    return normalize


def get_train_transform(IMAGE_SIZE, pretrained):
    """
    Returns the train transformation pipeline.
    """

    return transforms.Compose([
        transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
        # transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        # normalize_transform(pretrained),
    ])




# Validation transforms
def get_valid_transform(IMAGE_SIZE, pretrained):
    """
    Returns the validation transformation pipeline.
    """
    return transforms.Compose(
        [
            transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
            transforms.ToTensor(),
            # normalize_transform(pretrained),
        ]
    )



def get_datasets(pretrained, vit_processor_name=None):
    """
    Loads the train, validation, and test datasets.

    :param pretrained: Boolean, True or False, used to determine transformation type.
    :param vit_processor_name: Optional name of a processor model, used when loading a pretrained processor.

    Returns:
        tuple: The train, validation, and test datasets along with the class names.
    """
    # Set transformations based on whether pretrained and processor are specified
    train_transform = get_train_transform(IMAGE_SIZE, pretrained) if not vit_processor_name else None
    valid_transform = get_valid_transform(IMAGE_SIZE, pretrained) if not vit_processor_name else None

    # Load datasets with specified transformations
    dataset_train = datasets.ImageFolder(
        os.path.join(ROOT_DIR, "train"),
        transform=train_transform
    )
    dataset_valid = datasets.ImageFolder(
        os.path.join(ROOT_DIR, "validation"),
        transform=valid_transform
    )
    dataset_test = datasets.ImageFolder(
        os.path.join(ROOT_DIR, "test"),
        transform=valid_transform  # Using the same transform for validation and test
    )

    return dataset_train, dataset_valid, dataset_test, dataset_train.classes


def get_data_loaders(dataset_train, dataset_valid):
    train_loader = DataLoader(
        dataset_train,
        batch_size=BATCH_SIZE,
        shuffle=True,
        num_workers=NUM_WORKERS,
        pin_memory=True,
        prefetch_factor=4,  # Increased for better utilization
        persistent_workers=True,
    )
    valid_loader = DataLoader(
        dataset_valid,
        batch_size=BATCH_SIZE,
        shuffle=False,
        num_workers=NUM_WORKERS,
        pin_memory=True,
        prefetch_factor=4,
        persistent_workers=True
    )
    return train_loader, valid_loader


# Enable mixed precision and cuDNN optimizations
scaler = GradScaler()
torch.backends.cudnn.benchmark = True
torch.backends.cudnn.enabled = True

def find_optimal_lr(model, train_loader, criterion, optimizer, device="cuda", vit=None):
    lrs = []
    losses = []
    lr = 1e-7  # Starting small
    optimizer.param_groups[0]["lr"] = lr

    # Iterate through the training set to test different learning rates
    for i, batch in tqdm(enumerate(train_loader), total=len(train_loader)):

        optimizer.zero_grad()

        batch['pixel_values'] = batch['pixel_values'].to(device)
        batch['labels'] = batch['labels'].to(device)

        with autocast(device):
              outputs = model(**batch)
              loss = criterion(outputs.logits, batch['labels'])

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        # Record the learning rate and loss
        lrs.append(lr)
        losses.append(loss.item())

        # Increase the learning rate
        lr *= 1.1
        optimizer.param_groups[0]["lr"] = lr
        if lr > 1:  # Avoid too high learning rates
            break

    # Plot Loss vs Learning Rate
    plt.plot(lrs, losses)
    plt.xscale("log")
    plt.xlabel("Learning Rate")
    plt.ylabel("Loss")
    plt.title("Optimal Learning Rate Finder")
    plt.show()

    optimal_lr = lrs[np.argmin(losses)]
    print(f"Suggested Optimal Learning Rate: {optimal_lr}")
    return optimal_lr



def train(
    model,
    train_loader,
    optimizer,
    criterion,
    epoch,
    scheduler,
    early_stopping,
    pretrained,
    checkpoint_interval=10,
    device="cuda",
):
    model.train()
    train_running_loss = 0.0
    train_running_correct = 0
    counter = 0

    for i, batch in tqdm(enumerate(train_loader), total=len(train_loader)):
        counter += 1
        optimizer.zero_grad()

        batch['pixel_values'] = batch['pixel_values'].to(device)
        batch['labels'] = batch['labels'].to(device)

        with autocast(device):
              outputs = model(**batch)
              loss = criterion(outputs.logits, batch['labels'])

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        train_running_loss += loss.item()
        _, preds = torch.max(outputs.logits, 1)
        train_running_correct += (preds == batch['labels']).sum().item()

    epoch_loss = train_running_loss / counter
    epoch_acc = 100.0 * (train_running_correct / len(train_loader.dataset))
    print(f"Epoch [{epoch}] - Train Loss: {epoch_loss:.4f}, Train Acc: {epoch_acc:.2f}%")

    # Scheduler step
    scheduler.step(epoch_loss)

    return epoch_loss, epoch_acc


def validate(model, valid_loader, criterion, device="cuda"):
    model.eval()
    valid_running_loss = 0.0
    valid_running_correct = 0
    counter = 0

    with torch.no_grad():
        for i, batch in tqdm(enumerate(valid_loader), total=len(valid_loader)):
            counter += 1

            batch['pixel_values'] = batch['pixel_values'].to(device)
            batch['labels'] = batch['labels'].to(device)

            with autocast(device):
                  outputs = model(**batch)
                  loss = criterion(outputs.logits, batch['labels'])
            valid_running_loss += loss.item()

            _, preds = torch.max(outputs.logits, 1)
            valid_running_correct += (preds == batch['labels']).sum().item()

    epoch_loss = valid_running_loss / counter
    epoch_acc = 100.0 * (valid_running_correct / len(valid_loader.dataset))
    print(f"Validation Loss: {epoch_loss:.4f}, Validation Acc: {epoch_acc:.2f}%")

    return epoch_loss, epoch_acc

def get_optimizer_and_scheduler(model, optimal_lr):
    optimizer = torch.optim.AdamW(model.parameters(), lr=optimal_lr, weight_decay=1e-4)
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5, verbose=True)
    return optimizer, scheduler



In [ ]:
def build_model(pretrained=True, fine_tune=True, num_classes=10):
    if pretrained:
        print("[INFO]: Loading pre-trained weights")
    else:
        print("[INFO]: Not loading pre-trained weights")

    # Load the pre-trained EfficientNet-B0 model
    model = models.efficientnet_b0(
        weights=EfficientNet_B0_Weights.DEFAULT if pretrained else None
    )

    if not fine_tune:
        print("[INFO]: Freezing hidden layers...")
        for param in model.parameters():
            param.requires_grad = False
    elif fine_tune == True:
        print("[INFO]: Unfreezing all layers...")
        for param in model.parameters():
            param.requires_grad = True

    # Change the final classification head
    num_ftrs = model.classifier[1].in_features
    model.classifier[1] = nn.Linear(num_ftrs, num_classes)

    return model

In [ ]:
# Load the training and validation datasets.
dataset_train, dataset_test, dataset_valid, dataset_classes = get_datasets(
    pretrained=False
)
print(f"[INFO]: Number of training images: {len(dataset_train)}")
print(f"[INFO]: Number of validation images: {len(dataset_valid)}")
print(f"[INFO]: Class names: {dataset_classes}\n")
# Load the training and validation data loaders.
train_loader, valid_loader = get_data_loaders(dataset_train, dataset_valid)
# Learning_parameters.

epochs = 50
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Computation device: {device}")

print(f"Epochs to train for: {epochs}\n")
model = build_model(
    pretrained=True, fine_tune=True, num_classes=len(dataset_classes)
).to(device)

# Total parameters and trainable parameters.
total_params = sum(p.numel() for p in model.parameters())
print(f"{total_params:,} total parameters.")
total_trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"{total_trainable_params:,} training parameters.")

# Optimizer.
optimizer = optim.AdamW(model.parameters(), lr=0.002, weight_decay=1e-4)
# Loss function.
criterion = nn.CrossEntropyLoss()


# Lists to keep track of losses and accuracies.
train_losses, valid_losses = [], []
train_accs, valid_accs = [], []

optimal_lr = find_optimal_lr(model, train_loader, criterion, optimizer, device)
optimizer, scheduler = get_optimizer_and_scheduler(model, optimal_lr)

# Initialize early stopping
early_stopping = EarlyStopping(patience=10, verbose=True)

# Main Training Loop
for epoch in range(epochs):
    train_loss, train_acc = train(
        model,
        train_loader,
        optimizer,
        criterion,
        epoch,
        scheduler,
        early_stopping,
        pretrained=True,
        device=device,
    )

    val_loss, val_acc = validate(model, valid_loader, criterion, device=device)

    # Early stopping
    early_stopping(val_loss, model)
    if early_stopping.early_stop:
        print("Early stopping triggered!")
        break

    train_losses.append(train_loss)
    valid_losses.append(val_loss)
    train_accs.append(train_acc)
    valid_accs.append(val_acc)
    print(f"Training loss: {train_loss:.3f}, accuracy: {train_acc:.3f}%")
    print(f"Validation loss: {val_loss:.3f}, accuracy: {val_acc:.3f}%")

In [ ]:
# plot train, val loss acc over epoch
save_plots(train_accs, valid_accs, train_losses, valid_losses, pretrained=True)

# ***Vision Transformer - Transfer Learning***

In [6]:
from transformers import ViTImageProcessor, ViTForImageClassification

In [7]:
processor = ViTImageProcessor.from_pretrained("google/vit-base-patch16-224")
model = ViTForImageClassification.from_pretrained("google/vit-base-patch16-224")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/69.7k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

In [ ]:
!pip install evaluate

In [9]:
import evaluate

In [10]:
# K for top accuracy metric
k_for_top_acc = 1

# Compute metrics function for binary classification
acc_metric = evaluate.load("accuracy", module_type="metric")

def compute_metrics(eval_pred):
    predicted_probs, labels = eval_pred
    # Accuracy
    predicted_labels = np.argmax(predicted_probs, axis=1)
    acc = acc_metric.compute(predictions=predicted_labels, references=labels)
    # Top-K Accuracy
    top_k_indexes = [np.argpartition(row, -k_for_top_acc)[-k_for_top_acc:] for row in predicted_probs]
    top_k_classes = [top_k_indexes[i][np.argsort(row[top_k_indexes[i]])] for i, row in enumerate(predicted_probs)]
    top_k_classes = np.flip(np.array(top_k_classes), 1)
    acc_k = {
        f"accuracy_k" : sum([label in predictions for predictions, label in zip(top_k_classes, labels)]) / len(labels)
    }
    # Merge metrics
    acc.update(acc_k)
    return acc

## Vit


In [8]:
model.to(device)

ViTForImageClassification(
  (vit): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTSdpaAttention(
            (attention): ViTSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_fe

In [6]:
# Data loading function
def collate_fn(batch):
    images, labels = zip(*batch)
    inputs = processor(images=list(images), return_tensors="pt")
    inputs['labels'] = torch.tensor(labels)
    return inputs

In [11]:

dataset_train, dataset_valid, dataset_test, dataset_classes = get_datasets(
    pretrained=True, vit_processor_name="google/vit-base-patch16-224"
)

# train_loader = DataLoader(
#         dataset_train,
#         batch_size=BATCH_SIZE,
#         shuffle=True,
#         num_workers=NUM_WORKERS,
#         pin_memory=True,
#         prefetch_factor=4,  # Increased for better utilization
#         persistent_workers=True,
#         collate_fn=collate_fn
# )
# valid_loader = DataLoader(
#     dataset_valid,
#     batch_size=BATCH_SIZE,
#     shuffle=False,
#     num_workers=NUM_WORKERS,
#     pin_memory=True,
#     prefetch_factor=4,
#     persistent_workers=True,
#     collate_fn=collate_fn
# )

id2label = {i: c for i, c in enumerate(dataset_classes)}
label2id = {c: i for i, c in enumerate(dataset_classes)}

model.config.id2label = id2label
model.config.label2id = label2id


In [44]:
# Optimizer.
optimizer = optim.AdamW(model.parameters(), lr=0.002, weight_decay=1e-4)
# Loss function.
criterion = nn.CrossEntropyLoss()

In [11]:
model_dir = "model"
output_data_dir = "outputs"

# Total number of training epochs to perform
num_train_epochs = 15
# The batch size per GPU/TPU core/CPU for training
per_device_train_batch_size = 64
# The batch size per GPU/TPU core/CPU for evaluation
per_device_eval_batch_size = 64
# The initial learning rate for AdamW optimizer
learning_rate = 2e-5
# Number of steps used for a linear warmup from 0 to learning_rate
warmup_steps = 500
# The weight decay to apply to all layers except all bias and LayerNorm weights in AdamW optimizer
weight_decay = 0.01

main_metric_for_evaluation = "accuracy"

In [46]:
import gc
torch.cuda.empty_cache()
gc.collect()

2964

In [12]:
from transformers import TrainingArguments, Trainer

In [23]:
# Define training args
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir=model_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    per_device_eval_batch_size=per_device_eval_batch_size,
    warmup_steps=warmup_steps,
    weight_decay=weight_decay,
    eval_strategy="epoch",
    save_strategy="epoch",  # Consider "no" for save_strategy if not needed
    logging_strategy="steps",  # Log per few steps instead of every epoch for better visibility
    logging_steps=50,  # Customize based on how frequently you want logs (e.g., every 50 steps)
    logging_dir=f"{output_data_dir}/logs",
    learning_rate=float(learning_rate),
    remove_unused_columns=False,  # Needed for custom datasets, set to True if columns are compatible
    push_to_hub=False,
    report_to="tensorboard",
    load_best_model_at_end=True,
    metric_for_best_model=main_metric_for_evaluation,

    # Optimizations for faster training
    fp16=True,                   # Mixed precision training for faster performance on GPU
    bf16=False,                  # Use `bf16=True` if on a GPU supporting bf16 (e.g., A100)
    gradient_checkpointing=True, # Reduces memory use but adds slight compute overhead
    dataloader_num_workers=8,    # Adjust based on GPU and CPU cores available (4-8 common for large GPUs)
    dataloader_prefetch_factor=4,

    # Reduce checkpoint frequency for efficiency
    save_total_limit=2,          # Keep only the last 2 checkpoints



)

# Create Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    train_dataset=dataset_train,
    eval_dataset=dataset_valid,
    tokenizer=processor,
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [24]:
results = trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Epoch,Training Loss,Validation Loss,Accuracy,Accuracy K
1,1.119500,0.854363,0.718528,0.718773
2,0.630500,0.566655,0.802699,0.802699
3,0.413200,0.509923,0.824294,0.824785
4,0.282900,0.495653,0.839264,0.839018
5,0.153000,0.499713,0.849325,0.849080
6,0.068500,0.542822,0.853742,0.853742
7,0.046900,0.588824,0.855460,0.855460
8,0.038500,0.625153,0.858405,0.858160
9,0.019800,0.652241,0.859387,0.859387
10,0.012400,0.657064,0.863313,0.863313


/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/

# ***EfficientNet - Transfer Learning***

In [34]:
import gc
torch.cuda.empty_cache()
gc.collect()


0

In [11]:
import timm

In [12]:
model = timm.create_model('tf_efficientnetv2_m.in21k_ft_in1k', pretrained=True, num_classes=10)

model.safetensors:   0%|          | 0.00/218M [00:00<?, ?B/s]

In [18]:
dataset_train, dataset_valid, dataset_test, dataset_classes = get_datasets(
    pretrained=True, vit_processor_name="google/vit-base-patch16-224"
)

id2label = {i: c for i, c in enumerate(dataset_classes)}
label2id = {c: i for i, c in enumerate(dataset_classes)}

In [ ]:
model.to(device)

In [29]:
# Data loading function for cnn model
def collate_fn_for_cnn(batch):
    images, labels = zip(*batch)
    inputs = processor(images=list(images), return_tensors="pt")
    inputs['labels'] = torch.tensor(labels)
    return inputs


In [45]:
from transformers import TrainingArguments, Trainer
import torch
from torch import nn
from typing import Dict, Union, Any, Optional, List, Tuple

class CNNTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(inputs['pixel_values'])
        loss = nn.CrossEntropyLoss()(outputs, labels)

        return loss


    def prediction_step(
        self,
        model: nn.Module,
        inputs: Dict[str, Union[torch.Tensor, Any]],
        prediction_loss_only: bool,
        ignore_keys: Optional[List[str]] = None,
    ) -> Tuple[Optional[torch.Tensor], Optional[torch.Tensor], Optional[torch.Tensor]]:
        """
        Perform an evaluation step on `model` using `inputs` for EfficientNet.

        Args:
            model (`nn.Module`): The EfficientNet model to evaluate.
            inputs (`Dict[str, Union[torch.Tensor, Any]]`): The inputs and targets of the model.
            prediction_loss_only (`bool`): Whether or not to return the loss only.
            ignore_keys (`List[str]`, *optional*): A list of keys in the output of your model to be ignored.

        Return:
            Tuple[Optional[torch.Tensor], Optional[torch.Tensor], Optional[torch.Tensor]]:
            A tuple with the loss, logits, and labels (each being optional).
        """
        # Check if labels are provided
        has_labels = 'labels' in inputs and inputs['labels'] is not None
        inputs = self._prepare_inputs(inputs)

        # Get the labels if available
        labels = inputs.pop('labels', None) if has_labels else None

        # Handling ignore_keys (though it's unlikely needed for EfficientNet)
        if ignore_keys is None:
            ignore_keys = []

        with torch.no_grad():
            # Handle loss computation and inference
            if has_labels:
                # Forward pass with labels
                outputs = model(inputs['pixel_values'])

                # Compute loss
                loss_fn = nn.CrossEntropyLoss().to(device)
                loss = loss_fn(outputs, labels)

                # Logits are directly the outputs
                logits = outputs
            else:
                loss = None
                # Inference without labels
                outputs = model(inputs['pixel_values'])
                logits = outputs

        # If only loss is required
        if prediction_loss_only:
            return (loss, None, None)

        # Detach logits and labels for CPU transfer
        logits = logits.detach().cpu()
        if labels is not None:
            labels = labels.detach().cpu()

        return (loss, logits, labels)

In [46]:
model_dir = "model_cnn"
output_data_dir = "outputs_cnn"

# Total number of training epochs to perform
num_train_epochs = 15
# The batch size per GPU/TPU core/CPU for training
per_device_train_batch_size = 64
# The batch size per GPU/TPU core/CPU for evaluation
per_device_eval_batch_size = 64
# The initial learning rate for AdamW optimizer
learning_rate = 2e-5
# Number of steps used for a linear warmup from 0 to learning_rate
warmup_steps = 500
# The weight decay to apply to all layers except all bias and LayerNorm weights in AdamW optimizer
weight_decay = 0.01

main_metric_for_evaluation = "accuracy"

training_args_for_cnn = TrainingArguments(
    output_dir=model_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    per_device_eval_batch_size=per_device_eval_batch_size,
    warmup_steps=warmup_steps,
    weight_decay=weight_decay,
    eval_strategy="epoch",
    save_strategy="epoch",  # Consider "no" for save_strategy if not needed
    logging_strategy="steps",  # Log per few steps instead of every epoch for better visibility
    logging_steps=50,  # Customize based on how frequently you want logs (e.g., every 50 steps)
    logging_dir=f"{output_data_dir}/logs",
    learning_rate=float(learning_rate),
    remove_unused_columns=False,  # Needed for custom datasets, set to True if columns are compatible
    push_to_hub=False,
    report_to="tensorboard",
    load_best_model_at_end=True,
    metric_for_best_model=main_metric_for_evaluation,

    # Optimizations for faster training
    fp16=True,                   # Mixed precision training for faster performance on GPU
    bf16=False,                  # Use `bf16=True` if on a GPU supporting bf16 (e.g., A100)
    dataloader_num_workers=8,    # Adjust based on GPU and CPU cores available (4-8 common for large GPUs)
    dataloader_prefetch_factor=4,

    # Reduce checkpoint frequency for efficiency
    save_total_limit=2,          # Keep only the last 2 checkpoints



)

# Create Trainer instance
trainer_for_cnn = CNNTrainer(
    model=model,
    args=training_args_for_cnn,
    data_collator=collate_fn_for_cnn,
    compute_metrics=compute_metrics,
    train_dataset=dataset_train,
    eval_dataset=dataset_valid,
    tokenizer=processor,
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [47]:
results = trainer_for_cnn.train()

Epoch,Training Loss,Validation Loss,Accuracy,Accuracy K
1,0.003600,1.548442,0.677791,0.677791
2,0.467200,1.021397,0.734233,0.734233
3,0.167500,1.061417,0.758528,0.758528
4,0.122300,1.060892,0.770307,0.770307
5,0.073400,1.064850,0.776933,0.777178
6,0.038700,1.085917,0.779387,0.779141
7,0.035400,1.073976,0.782086,0.782086
8,0.036500,1.092684,0.785276,0.785031
9,0.020300,1.104457,0.787239,0.787239
10,0.012700,1.105941,0.791656,0.792147
